## normalized burn ratio


In [1]:
workflow = dict([('id', 'nbr'),
                ('label', 'Normalized burn ratio'),
                ('doc', 'Normalized burn ratio processor')])


In [2]:
input_reference = dict([('id', 'input_reference'), 
                        ('label', 'EO product for normalized burn ratio'),
                        ('doc', 'EO product for normalized burn ratio'),
                        ('value', 'https://catalog.terradue.com/sentinel2/search?uid=S2B_MSIL2A_20190612T095039_N0212_R079_T33TVF_20190612T124245'), 
                        ('stac:collection', 'input_reference'),
                        ('stac:href', '/workspace/data/s2/catalog.json'),
                        ('type', 'Directory'),
                        ('scatter', 'True')])

### Vegetation indexes

NBR = (NIR - SWIR22) / (NIR + SWIR22)
NDVI = (NIR - Red) / (NIR + Red)
NDWI = (NIR - SWIR16) / (NIR + SWIR16)

In [3]:
bands = [{'name': 'NBR',
          'common_name': 'nbr'}, 
         {'name': 'NDVI',
          'common_name': 'ndvi'},
         {'name': 'NDWI',
          'common_name': 'ndwi'},
        ]

In [4]:
import os
import sys
import gdal
import numpy as np
import logging
from pystac import Catalog, Collection, EOItem, MediaType, EOAsset, CatalogType
from time import sleep

gdal.UseExceptions()

if not 'PREFIX' in os.environ.keys():
    
    os.environ['PREFIX'] = '/opt/anaconda/envs/env_nbr/'

os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
cat = Catalog.from_file(input_reference['stac:href'])

In [7]:
collection = next(cat.get_children())

In [8]:
item = next(collection.get_items())

In [9]:
for index, band in enumerate(item.bands):
   
    if band.common_name in ['swir16']:
 
       asset_swir16 = item.assets[band.name].get_absolute_href()

    if band.common_name in ['swir22']:
 
       asset_swir22 = item.assets[band.name].get_absolute_href()
        
    if band.common_name in ['nir']:
 
        asset_nir = item.assets[band.name].get_absolute_href()
    
    if band.common_name in ['red']:
 
        asset_red = item.assets[band.name].get_absolute_href()

In [10]:
vrt = '{0}.vrt'.format(item.id)

In [11]:
ds = gdal.BuildVRT(vrt,
                   [asset_red, asset_nir, asset_swir16, asset_swir22],
                   srcNodata=0,
                   xRes=10, 
                   yRes=10,
                   separate=True)

ds.FlushCache()

ds = None

del(ds)


In [12]:
tif = '{0}.tif'.format(item.id)

In [13]:
gdal.Translate(tif,
               vrt,
               outputType=gdal.GDT_Int16)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f91909e1990> >

In [14]:
os.remove(vrt)

In [15]:
ds = gdal.Open(tif)
width = ds.RasterXSize
height = ds.RasterYSize

input_geotransform = ds.GetGeoTransform()
input_georef = ds.GetProjectionRef()

In [16]:
red = ds.GetRasterBand(1).ReadAsArray()
nir = ds.GetRasterBand(2).ReadAsArray()
swir16 = ds.GetRasterBand(3).ReadAsArray()
swir22 = ds.GetRasterBand(4).ReadAsArray()

In [17]:
nbr = np.zeros((height, width), dtype=np.uint)

nbr = 10000 * ((nir - swir22) / (nir + swir22))

/opt/anaconda/envs/env_nbr/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
swir22 = None

In [19]:
ndvi = np.zeros((height, width), dtype=np.uint)

ndvi = 10000 * ((nir - red) / (nir + red))

/opt/anaconda/envs/env_nbr/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
red = None

In [21]:
ndwi = np.zeros((height, width), dtype=np.uint)

ndwi = 10000 * ((nir - swir16) / (nir + swir16))

/opt/anaconda/envs/env_nbr/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
nir = swir16 = None

In [23]:
#temp_name = '_NRB_{}.tif'.format(item.id)
#output_name = 'NRB_{}.tif'.format(item.id)

In [24]:
def cog(input_tif, output_tif,no_data=None):
    
    translate_options = gdal.TranslateOptions(gdal.ParseCommandLine('-co TILED=YES ' \
                                                                    '-co COPY_SRC_OVERVIEWS=YES ' \
                                                                    '-co COMPRESS=LZW '))
    
    if no_data != None:
        translate_options = gdal.TranslateOptions(gdal.ParseCommandLine('-co TILED=YES ' \
                                                                        '-co COPY_SRC_OVERVIEWS=YES ' \
                                                                        '-co COMPRESS=LZW '\
                                                                        '-a_nodata {}'.format(no_data)))
    ds = gdal.Open(input_tif, gdal.OF_READONLY)

    gdal.SetConfigOption('COMPRESS_OVERVIEW', 'DEFLATE')
    ds.BuildOverviews('NEAREST', [2,4,8,16,32])
    
    ds = None

    ds = gdal.Open(input_tif)
    gdal.Translate(output_tif,
                   ds, 
                   options=translate_options)
    ds = None

    os.remove('{}.ovr'.format(input_tif))
    os.remove(input_tif)


In [25]:
catalog = Catalog(id='catalog', description='Results')

catalog.clear_items()
catalog.clear_children()

<Catalog id=catalog>

In [26]:
item_name = 'INDEX_{}'.format(item.id)

In [27]:
result_item = EOItem(id=item_name,
                   geometry=item.geometry,
                   bbox=item.bbox,
                   datetime=item.datetime,
                   properties={},
                   bands=bands,
                    gsd=10, 
                    platform=item.platform, 
                    instrument=item.instrument)

In [28]:
for index, veg_index in enumerate(['NBR', 'NDVI', 'NDWI']):

    temp_name = '_{}_{}.tif'.format(veg_index, item.id)
    output_name = '{}_{}.tif'.format(veg_index, item.id)

    driver = gdal.GetDriverByName('GTiff')
    
    output = driver.Create(temp_name, 
                           width, 
                           height, 
                           1, 
                           gdal.GDT_Int16)

    output.SetGeoTransform(input_geotransform)
    output.SetProjection(input_georef)
    output.GetRasterBand(1).WriteArray(nbr),

    output.FlushCache()

    sleep(5)
    
    output = None

    del(output)

    os.makedirs(os.path.join('stac-results', item_name),
                exist_ok=True)
    
    cog(temp_name, os.path.join('stac-results', item_name, output_name))

    result_item.add_asset(key=veg_index.lower(),
                          asset=EOAsset(href='./{}'.format(output_name), 
                          media_type=MediaType.GEOTIFF, 
                          title=bands[index]['name'],
                          bands=bands[index]))

In [29]:
catalog.add_items([result_item])

In [30]:
os.remove(tif)

In [31]:
catalog.normalize_and_save(root_href='stac-results',
                           catalog_type=CatalogType.SELF_CONTAINED)

In [32]:
catalog.describe()

* <Catalog id=catalog>
  * <EOItem id=INDEX_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>
